In [1]:
%cd /data/nijhawanlab/maria/
R1_file = "NB_R1_001.fastq.gz"
R2_file = "NB_R2_001.fastq.gz"
primer_file = "primers.2.tsv"

/data/nijhawanlab/maria


In [2]:
from Bio import Seq, SeqIO, Align, Alphabet
# import numpy as np
# import pandas as pd
# import seaborn as sns
# import matplotlib.pyplot as plt
import gzip
import csv
import itertools
import re
import pprint

In [3]:
basePair = {'A':'T','T':'A','G':'C','C':'G'}
basePairTrans = str.maketrans(basePair)
def rcDNA(seq):
    return seq.translate(basePairTrans) [::-1]
def cDNA(seq):
    return seq.translate(basePairTrans)

In [4]:
primers = []
with open(primer_file, newline='') as primers_IO:
    primer_reader = csv.DictReader(primers_IO,
                                   delimiter='\t',
                                   fieldnames=['tag','forward_primer','reverse_primer'])
    for primer in primer_reader:
        primers.append({
            'tag':primer['tag'],
            'forward_primer':primer['forward_primer'].upper(),
            'reverse_primer':primer['reverse_primer'].upper(),
        })

primers

[{'tag': 'BC1',
  'forward_primer': 'AGTTGTTATGCAAATGGAAGGTGT',
  'reverse_primer': 'TCACGCTCAGAAAGATAACAAAGCAAGACA'},
 {'tag': 'BC2',
  'forward_primer': 'AGTTGTTATGCAAATGGAAGGTGT',
  'reverse_primer': 'AAAACTGAGCCAGCCCTTATCCTCTTAA'},
 {'tag': 'BC3',
  'forward_primer': 'AGTTGTTATGCAAATGGAAGGTGT',
  'reverse_primer': 'TCACGCCAGAAAGATAACAAAGCAAGACA'},
 {'tag': 'BC4',
  'forward_primer': 'AGTTGTTATGCAAATGGAAGGTGT',
  'reverse_primer': 'TCACAGAAAGATGAAAACAAAGCAAGACA'},
 {'tag': 'BC5',
  'forward_primer': 'AGTTGTTATGCAAATGGAAGGTGT',
  'reverse_primer': 'TCACAGATGAAAGATAACAAAGCAAGACA'},
 {'tag': 'BC6',
  'forward_primer': 'AGTTGTTATGCAAATGGAAGGTGT',
  'reverse_primer': 'TCACGGTAGAAAGATAACAAAGCAAGACA'}]

In [5]:
def matchSinglePrimer(seq, primer_seq):
    direction = 1
    pos = seq.find(primer_seq)
    if pos < 0:
        pos = seq.find( rcDNA(primer_seq) )
        direction = -1
    return pos, direction

def findPrimers(seq, primers):
    primer_matches = {'forward':[],'reverse':[]}
    for p in primers:
        rev_pos, rev_dir = matchSinglePrimer(seq, p['reverse_primer'])
        # print(rev_pos)
        if rev_pos >= 0:
            primer_matches['reverse'].append( (p['tag'], rev_pos) )
        fwd_pos, fwd_dir = matchSinglePrimer(seq, p['forward_primer'])
        # print(fwd_pos)
        if fwd_pos >= 0:
            primer_matches['forward'].append( (p['tag'], fwd_pos) )
    return primer_matches

In [6]:
maxReads = 300000
readNumber = 0
aligning = False

pp = pprint.PrettyPrinter(indent=4)

read_stats = {
    p['tag']: 0
    for p in primers
}
read_stats['unassigned'] = 0
read_stats['total'] = 0

if aligning:
    align_file = open("alignments.txt", 'w')

split_files = {}
for p in primers:
    tag = p['tag']
    R1_filename = "{}_R1.fastq".format(tag)
    R2_filename = "{}_R2.fastq".format(tag)
    split_files[tag]={'R1': open(R1_filename, 'w'), 'R2': open(R2_filename, 'w')}
    
with gzip.open(R1_file, "rt") as R1, gzip.open(R2_file, "rt") as R2:
    reads1 = SeqIO.parse(R1, "fastq")
    reads2 = SeqIO.parse(R2, "fastq")
    for (read1, read2) in zip(reads1, reads2):
        R2_sequence = str(read2.seq)
        R1_sequence = str(read1.seq)

        if aligning:
            aligner = Align.PairwiseAligner()
            alignments = aligner.align(R1_sequence, rcDNA( R2_sequence ))
            alignment = alignments[0]
            align_file.write(alignment.format())
            align_file.write('\n')
        
        matches['R1'] = findPrimers(R1_sequence, primers)
        matches['R2'] = findPrimers(R2_sequence, primers)
        forward_primer_set = {p for (p, pos) in matches['R1']['forward']}.union(
            {p for (p, pos) in matches['R2']['forward']}
        )
        reverse_primer_set = {p for (p, pos) in matches['R1']['reverse']}.union(
            {p for (p, pos) in matches['R2']['reverse']}
        )
        possible_primer_set = reverse_primer_set.intersection(forward_primer_set)
        # print(list(forward_primer_set))
        # print(list(reverse_primer_set))
        # print(list(possible_primer_set))

        if len(possible_primer_set) == 1:
            the_primer = possible_primer_set.pop()
            read_stats[the_primer] += 1
            SeqIO.write(read1, split_files[the_primer]['R1'], "fastq") 
            SeqIO.write(read2, split_files[the_primer]['R2'], "fastq")
        else:
            read_stats['unassigned'] += 1
        read_stats['total'] += 1
        
        # print("R1: {}/{}\t R2: {}/{}".format(
        #     matches['R1']['forward'], matches['R1']['reverse'],
        #     matches['R2']['forward'], matches['R2']['reverse'],
        # ))
        
        readNumber += 1
        if readNumber >= maxReads:
            break

if aligning:
    align_file.close()
    
print("maxReads: {}\nTotal reads: {}".format(maxReads,readNumber))
pp.pprint(read_stats)

NameError: name 'matches' is not defined

In [7]:
maxReads = 35
readNumber = 0
aligning = False

pp = pprint.PrettyPrinter(indent=4)

read_stats = {
    p['tag']: 0
    for p in primers
}
read_stats['unassigned'] = 0
read_stats['total'] = 0
R1_records = []
R2_records = []

with gzip.open(R1_file, "rt") as R1, gzip.open(R2_file, "rt") as R2:
    reads1 = SeqIO.parse(R1, "fastq")
    reads2 = SeqIO.parse(R2, "fastq")
    for (read1, read2) in zip(reads1, reads2):
        R1_records.append(read1)
        R2_records.append(read2)
        
        readNumber += 1
        if readNumber >= maxReads:
            break

print("maxReads: {}\nTotal reads: {}".format(maxReads,readNumber))
pp.pprint(read_stats)

maxReads: 35
Total reads: 35
{   'BC1': 0,
    'BC2': 0,
    'BC3': 0,
    'BC4': 0,
    'BC5': 0,
    'BC6': 0,
    'total': 0,
    'unassigned': 0}


In [8]:
r = R1_records[5]
r

SeqRecord(seq=Seq('CACAGATGAAAGATAACAAAGCAAGACATACAGAAATTATGTAGTTATACTCAG...TAC', SingleLetterAlphabet()), id='GWNJ-1012:466:GW210102944th.Miseq:1:2101:12608:1188', name='GWNJ-1012:466:GW210102944th.Miseq:1:2101:12608:1188', description='GWNJ-1012:466:GW210102944th.Miseq:1:2101:12608:1188 1:N:0:GCTCATGA+CGGAGAGA', dbxrefs=[])

In [9]:
r.letter_annotations

{'phred_quality': [37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  25,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  25,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  25,
  37,
  37,
  37,
  37,
  37,
  37,

In [10]:
primers[0]['forward_primer']
p = Seq.Seq(primers[0]['forward_primer'])
p

Seq('AGTTGTTATGCAAATGGAAGGTGT')

In [11]:
r = R1_records[8]
(p in r,
p.complement() in r,
p.reverse_complement() in r)

(True, False, False)

In [51]:
# Need to keep .find-ing if the quality is too low.

def findSubsequence(subSeq, fullSeq, min_quality=30):
    pos = fullSeq.seq.find(subSeq)
    while (pos < len(subSeq)) & (pos > -1):
        qualities = fullSeq.letter_annotations['phred_quality'][pos:pos+len(subSeq)]
        if all([q >= min_quality for q in qualities]):
            return (pos, qualities)
        pos = fullSeq.seq.find(subSeq, pos+1)
    return None

def findAllSubsequences(subSeq, fullSeq, min_quality=30):
    matches = []
    start_pos = 0
    max_pos = len(fullSeq)
    while ((start_pos < max_pos) & (start_pos > -1)):
        start_pos = fullSeq.seq.find(subSeq, start_pos)
        if start_pos > -1:
            quality = min(fullSeq.letter_annotations['phred_quality'][start_pos:start_pos + len(subSeq)])
            if quality > min_quality:
                matches.append({'position':start_pos, 'quality':quality})
            start_pos += 1
    return matches

def findBestSubsequence(subSeq, fullSeq, min_quality=30):
    best_match = {'position':-1, 'quality':-1}
    start_pos = 0
    max_pos = len(fullSeq)
    while ((start_pos < max_pos) & (start_pos > -1)):
        start_pos = fullSeq.seq.find(subSeq, start_pos)
        if start_pos > -1:
            quality = min(fullSeq.letter_annotations['phred_quality'][start_pos:start_pos + len(subSeq)])
            if quality > best_match['quality']:
                best_match['position'] = start_pos
                best_match['quality'] = quality
            start_pos += 1
    if best_match['quality'] >= min_quality:
        return best_match
    else:
        return None

In [40]:
findSubsequence(p,r, min_quality=10)

(0,
 [37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  37,
  25,
  37,
  37,
  37,
  37,
  37,
  25,
  37,
  37,
  37,
  37,
  37,
  37,
  37])

In [14]:
findSubsequence(Seq.Seq('GAAA'), r, min_quality=30)

In [15]:
p.find('TT')

2

In [16]:
p.find('TT',100)

-1

In [17]:
p.find('GAAG')

16

In [18]:
p.find('GAAN')

-1

In [41]:
findAllSubsequences(p, r, min_quality=10)

[{'position': 0, 'quality': 25}]

In [23]:
len(r)

250

In [43]:
findAllSubsequences('AAA', r, min_quality=10)

[{'position': 11, 'quality': 37},
 {'position': 32, 'quality': 37},
 {'position': 82, 'quality': 37},
 {'position': 83, 'quality': 37},
 {'position': 84, 'quality': 37},
 {'position': 85, 'quality': 25},
 {'position': 86, 'quality': 11},
 {'position': 107, 'quality': 25},
 {'position': 118, 'quality': 11},
 {'position': 119, 'quality': 11}]

In [44]:
findAllSubsequences('AAA', r, min_quality=20)

[{'position': 11, 'quality': 37},
 {'position': 32, 'quality': 37},
 {'position': 82, 'quality': 37},
 {'position': 83, 'quality': 37},
 {'position': 84, 'quality': 37},
 {'position': 85, 'quality': 25},
 {'position': 107, 'quality': 25}]

In [45]:
findAllSubsequences('AAA', r, min_quality=30)

[{'position': 11, 'quality': 37},
 {'position': 32, 'quality': 37},
 {'position': 82, 'quality': 37},
 {'position': 83, 'quality': 37},
 {'position': 84, 'quality': 37}]

In [46]:
findAllSubsequences('AAA', r, min_quality=40)

[]

In [53]:
findBestSubsequence('AAA', r, min_quality=10)

{'position': 11, 'quality': 37}

In [52]:
findBestSubsequence('ACTGACTG', r, min_quality=10)

In [128]:
read_groups = {}
with open(primer_file, newline='') as primers_IO:
    primer_reader = csv.DictReader(primers_IO,
                                   delimiter='\t',
                                   fieldnames=['tag','forward_primer','reverse_primer'])
    for primer in primer_reader:
        read_groups[primer['tag']] = {
            'forward_primer':primer['forward_primer'].upper(),
            'reverse_primer':primer['reverse_primer'].upper(),
            'read_count':0,
            'output_files':{'R1':None, 'R2':None}
        }

In [58]:
read_groups

{'BC1': {'forward_primer': 'AGTTGTTATGCAAATGGAAGGTGT',
  'reverse_primer': 'TCACGCTCAGAAAGATAACAAAGCAAGACA',
  'read_count': 0,
  'output_files': {'R1': None, 'R2': None}},
 'BC2': {'forward_primer': 'AGTTGTTATGCAAATGGAAGGTGT',
  'reverse_primer': 'AAAACTGAGCCAGCCCTTATCCTCTTAA',
  'read_count': 0,
  'output_files': {'R1': None, 'R2': None}},
 'BC3': {'forward_primer': 'AGTTGTTATGCAAATGGAAGGTGT',
  'reverse_primer': 'TCACGCCAGAAAGATAACAAAGCAAGACA',
  'read_count': 0,
  'output_files': {'R1': None, 'R2': None}},
 'BC4': {'forward_primer': 'AGTTGTTATGCAAATGGAAGGTGT',
  'reverse_primer': 'TCACAGAAAGATGAAAACAAAGCAAGACA',
  'read_count': 0,
  'output_files': {'R1': None, 'R2': None}},
 'BC5': {'forward_primer': 'AGTTGTTATGCAAATGGAAGGTGT',
  'reverse_primer': 'TCACAGATGAAAGATAACAAAGCAAGACA',
  'read_count': 0,
  'output_files': {'R1': None, 'R2': None}},
 'BC6': {'forward_primer': 'AGTTGTTATGCAAATGGAAGGTGT',
  'reverse_primer': 'TCACGGTAGAAAGATAACAAAGCAAGACA',
  'read_count': 0,
  'output_fil

In [54]:
r.id

'GWNJ-1012:466:GW210102944th.Miseq:1:2101:3088:1329'

In [55]:
def readCheckIDsMatch(read1, read2):
    return read1.id == read2.id

In [83]:
def openGroupFiles(read_groups):
    for k in read_groups:
        read_groups[k]['output_files']['R1'] = open(k + "_R1.fastq", "w")
        read_groups[k]['output_files']['R2'] = open(k + "_R2.fastq", "w")
        
def closeGroupFiles(read_groups):
    for k in read_groups:
        read_groups[k]['output_files']['R1'].close()
        read_groups[k]['output_files']['R2'].close()

In [148]:
import random

def assignReadsToGroupRandomly(read1, read2, read_groups):
    return random.choice( list(read_groups.keys()) )

def assignReadsToGroupByForwardPrimer(read1, read2, read_groups, quality=30):
    best_match = {'quality':0, 'group':''}
    for group in read_groups:
        m = findBestSubsequence(read_groups[group]['forward_primer'],
                                read1, min_quality=quality)
        if m != None:
            if m['quality'] > best_match['quality']:
                best_match['quality'] = m['quality']
                best_match['group'] = group
    for group in read_groups:
        m = findBestSubsequence(read_groups[group]['forward_primer'],
                                read2, min_quality=quality)
        if m != None:
            if m['quality'] > best_match['quality']:
                best_match['quality'] = m['quality']
                best_match['group'] = group
    if best_match['quality'] != 0:
        return best_match['group']
    else:
        return None
    
def assignReadsToGroupByReversePrimer(read1, read2, read_groups, quality=30):
    best_match = {'quality':0, 'group':''}
    for group in read_groups:
        m = findBestSubsequence(read_groups[group]['reverse_primer'],
                                read1, min_quality=quality)
        if m != None:
            if m['quality'] > best_match['quality']:
                best_match['quality'] = m['quality']
                best_match['group'] = group
    for group in read_groups:
        m = findBestSubsequence(read_groups[group]['reverse_primer'],
                                read2, min_quality=quality)
        if m != None:
            if m['quality'] > best_match['quality']:
                best_match['quality'] = m['quality']
                best_match['group'] = group
    if best_match['quality'] != 0:
        return best_match['group']
    else:
        return None

In [149]:
maxReads = 10000

pp = pprint.PrettyPrinter(indent=4)

prechecks = [readCheckIDsMatch]
assignReadsToGroup = assignReadsToGroupByReversePrimer

readNumber = 0
failed_reads = 0
unassigned_reads = 0

# open group files here
openGroupFiles(read_groups)

with gzip.open(R1_file, "rt") as R1, gzip.open(R2_file, "rt") as R2:
    reads1 = SeqIO.parse(R1, "fastq")
    reads2 = SeqIO.parse(R2, "fastq")
    for (read1, read2) in zip(reads1, reads2):
        readNumber += 1
        if readNumber >= maxReads:
            break

        # run pre-checks
        if any([check(read1, read2)==False for check in prechecks]):
            failed_reads += 1
            continue
            
        # assign group
        #group = assignReadsToGroupByForwardPrimer(read1, read2, read_groups)
        #if group == None:
        group = assignReadsToGroupByReversePrimer(read1, read2, read_groups, quality=10)
        if group == None:
            unassigned_reads += 1
            continue
        read_groups[group]['read_count'] += 1
        
        # output reads to group
        # print(group)
        SeqIO.write(read1, read_groups[group]['output_files']['R1'], "fastq") 
        SeqIO.write(read2, read_groups[group]['output_files']['R2'], "fastq") 
        

# close group files here
closeGroupFiles(read_groups)
            
print("maxReads: {}".format(maxReads))
print("Total reads: {}".format(readNumber))
print("failed reads: {}".format(failed_reads))
print("unassigned reads: {}".format(unassigned_reads))
pp.pprint(read_groups)

maxReads: 10000
Total reads: 10000
failed reads: 0
unassigned reads: 2852
{   'BC1': {   'forward_primer': 'AGTTGTTATGCAAATGGAAGGTGT',
               'output_files': {   'R1': <_io.TextIOWrapper name='BC1_R1.fastq' mode='w' encoding='ANSI_X3.4-1968'>,
                                   'R2': <_io.TextIOWrapper name='BC1_R2.fastq' mode='w' encoding='ANSI_X3.4-1968'>},
               'read_count': 9138,
               'reverse_primer': 'TCACGCTCAGAAAGATAACAAAGCAAGACA'},
    'BC2': {   'forward_primer': 'AGTTGTTATGCAAATGGAAGGTGT',
               'output_files': {   'R1': <_io.TextIOWrapper name='BC2_R1.fastq' mode='w' encoding='ANSI_X3.4-1968'>,
                                   'R2': <_io.TextIOWrapper name='BC2_R2.fastq' mode='w' encoding='ANSI_X3.4-1968'>},
               'read_count': 3076,
               'reverse_primer': 'AAAACTGAGCCAGCCCTTATCCTCTTAA'},
    'BC3': {   'forward_primer': 'AGTTGTTATGCAAATGGAAGGTGT',
               'output_files': {   'R1': <_io.TextIOWrapper name='BC3

In [76]:
random.choice(list(read_groups.keys()))

'BC4'

In [133]:
!pip install tabulate
import tabulate

You should consider upgrading via the '/opt/miniconda/envs/gatk/bin/python3.6 -m pip install --upgrade pip' command.


In [144]:
t = [[k, read_groups[k]['read_count']] for k in read_groups]
t.append(["rejected", failed_reads])
t.append(["unassigned", unassigned_reads])
t.append(["total", readNumber])
print(tabulate.tabulate(
    t,
    headers=["sample","read count"]
))

sample        read count
----------  ------------
BC1                 7297
BC2                 1195
BC3                  902
BC4                  262
BC5                  767
BC6                  596
failed                 0
unassigned          6274
total              10000


In [150]:
t = [[k, read_groups[k]['read_count']] for k in read_groups]
t.append(["failed", failed_reads])
t.append(["unassigned", unassigned_reads])
t.append(["total", readNumber])
print(tabulate.tabulate(
    t,
    headers=["sample","read count"]
))

sample        read count
----------  ------------
BC1                 9138
BC2                 3076
BC3                 2119
BC4                  646
BC5                 1789
BC6                 1398
failed                 0
unassigned          2852
total              10000


In [152]:
# read1
SeqIO.write(read1, None, "fastq") 

AttributeError: 'NoneType' object has no attribute 'write'